In [26]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException


# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#beautifulsoup
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint

import re

import json

import pandas as pd

# 加入行為鍊 ActionChain (在 WebDriver 中模擬滑鼠移動、點擊、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains

# 加入鍵盤功能 (例如 Ctrl、Alt 等)
from selenium.webdriver.common.keys import Keys



In [27]:
# 設定log檔案
import logging
logger = logging.getLogger('Taiwan_court_kill_cases_v2.log')
logger.setLevel(logging.INFO) 

# 設定輸出格式
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', "%Y-%m-%d %H:%M:%S")

# 儲存在 log 當中的事件處理器(存在.log檔中)
fileHandler = logging.FileHandler('Taiwan_court_kill_cases', mode='a', encoding='utf-8') # a: append, w: write  #建構方法
fileHandler.setFormatter(formatter)

# 輸出在終端機介面上的事件處理器
console_handler = logging.StreamHandler()  #建構方法
console_handler.setFormatter(formatter)    #將物件設定格式(這邊的formatter)

# 加入事件
logger.addHandler(console_handler)         #addHandler也就是說，這個logger物件要要如何處理log訊息
logger.addHandler(fileHandler)

In [28]:
# 處理日期的list 一組組搜尋寫成一個list這樣用index就可以搜尋了也比較好處理
all_split_date_combination_list = []
# 簡單生這樣就好[85,1,1, 6,30], [85,7,1,12,31]
for i in range(113,84, -1): # 民國113到85
    month_day_list = [[7,1,12,31], [1,1, 6,30]]
    for b in month_day_list:
        b.insert(0, i)
        all_split_date_combination_list.append(b)

In [29]:
all_split_date_combination_list

[[113, 7, 1, 12, 31],
 [113, 1, 1, 6, 30],
 [112, 7, 1, 12, 31],
 [112, 1, 1, 6, 30],
 [111, 7, 1, 12, 31],
 [111, 1, 1, 6, 30],
 [110, 7, 1, 12, 31],
 [110, 1, 1, 6, 30],
 [109, 7, 1, 12, 31],
 [109, 1, 1, 6, 30],
 [108, 7, 1, 12, 31],
 [108, 1, 1, 6, 30],
 [107, 7, 1, 12, 31],
 [107, 1, 1, 6, 30],
 [106, 7, 1, 12, 31],
 [106, 1, 1, 6, 30],
 [105, 7, 1, 12, 31],
 [105, 1, 1, 6, 30],
 [104, 7, 1, 12, 31],
 [104, 1, 1, 6, 30],
 [103, 7, 1, 12, 31],
 [103, 1, 1, 6, 30],
 [102, 7, 1, 12, 31],
 [102, 1, 1, 6, 30],
 [101, 7, 1, 12, 31],
 [101, 1, 1, 6, 30],
 [100, 7, 1, 12, 31],
 [100, 1, 1, 6, 30],
 [99, 7, 1, 12, 31],
 [99, 1, 1, 6, 30],
 [98, 7, 1, 12, 31],
 [98, 1, 1, 6, 30],
 [97, 7, 1, 12, 31],
 [97, 1, 1, 6, 30],
 [96, 7, 1, 12, 31],
 [96, 1, 1, 6, 30],
 [95, 7, 1, 12, 31],
 [95, 1, 1, 6, 30],
 [94, 7, 1, 12, 31],
 [94, 1, 1, 6, 30],
 [93, 7, 1, 12, 31],
 [93, 1, 1, 6, 30],
 [92, 7, 1, 12, 31],
 [92, 1, 1, 6, 30],
 [91, 7, 1, 12, 31],
 [91, 1, 1, 6, 30],
 [90, 7, 1, 12, 31],
 [90, 1,

In [30]:
driver = webdriver.Chrome()

In [31]:
# 放所有case的資料
all_case_list = []
# 每次查詢的總頁數網址
page_url_list = []
# 給每個case一個數字的primary key id
id_id = 1
# 每爬超過500筆，休息一下
rest_time_count = 1

In [32]:
# 現在遇到的問題是，網站訪問過多次，結果被拒絕存取了，那就試試看每500筆休5分鐘，看可不可以全部爬完拉

In [33]:
####這邊主程式

# 以切分日期的方式爬取所有資料
for date_split in all_split_date_combination_list:  #從113年取回來
    
    # 每爬超過500筆，休息一下
    if (len(all_case_list)/500) >= rest_time_count:
        logger.info(f"total cases:{len(all_case_list)}, rest 5 mins")
        sleep(300)
        rest_time_count += 1

    driver.get("https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx")
    # 搜尋
    try:
        WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
        (By.CSS_SELECTOR, 'input#jud_jmain')
        ))
        input_case_reason = driver.find_element(By.CSS_SELECTOR, 'input#jud_jmain')
        input_case_reason.clear()   
        input_case_reason.send_keys("殺人")
        
        input_case_content = driver.find_element(By.CSS_SELECTOR, 'input#jud_kw')
        input_case_content.clear()
        input_case_content.send_keys("中華民國刑法第 271 條&刑事判決")
        
        input_date_from_year = driver.find_element(By.CSS_SELECTOR, 'input#dy1')
        input_date_from_month = driver.find_element(By.CSS_SELECTOR, 'input#dm1')
        input_date_from_day = driver.find_element(By.CSS_SELECTOR, 'input#dd1')

        input_date_end_year = driver.find_element(By.CSS_SELECTOR, 'input#dy2')
        input_date_end_month = driver.find_element(By.CSS_SELECTOR, 'input#dm2')
        input_date_end_day = driver.find_element(By.CSS_SELECTOR, 'input#dd2')

        input_date_from_year.clear()
        input_date_from_month.clear()
        input_date_from_day.clear()
        input_date_end_year.clear()
        input_date_end_month.clear()
        input_date_end_day.clear()

        # 開始輸入數字
        ac = ActionChains(driver)
        ac.send_keys_to_element(input_date_from_year, date_split[0])
        ac.send_keys_to_element(input_date_from_month, date_split[1])
        ac.send_keys_to_element(input_date_from_day, date_split[2])
        ac.send_keys_to_element(input_date_end_year, date_split[0])
        ac.send_keys_to_element(input_date_end_month, date_split[3])
        ac.send_keys_to_element(input_date_end_day, date_split[4])


        # 執行
        ac.perform()
        sleep(1)
        search_button = driver.find_element(By.CSS_SELECTOR, 'input#btnQry')
        search_button.click()
        logger.info(f"{date_split} is search")
        sleep(1)
    except TimeoutException:
        logger.warning(f"{date_split} can not search")
        continue
    
    #等待iframe與下一頁按鈕，計算pages
    try:
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
        (By.CSS_SELECTOR, "iframe#iframe-data")
        ))

        iframe = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-data")
        one_page_url = iframe.get_attribute('src')  #切到iframe裡面就會抓不到了....要先抓....
        logger.info(f"search {date_split}, it's base url is {one_page_url}")
        driver.switch_to.frame(iframe)

        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
        (By.CSS_SELECTOR, "div#plPager > span")
        ))

        # 抓一次搜尋區間的筆數
        search_block_cases = driver.find_element(By.CSS_SELECTOR, "div#plPager > span")
        re_block_cases = r'(?<=共).*(?=筆)'
        block_cases_number = re.search(re_block_cases, search_block_cases.text)
        block_cases_number = int(block_cases_number[0])
        
    except TimeoutException:
        logger.info(f"{date_split} not found result, do next search") #若iframe抓不到表示沒有搜尋結果，就往下搜下一個就好
        continue                                                      # 還是會有iframe只是結果是error page，結論是不管這邊有沒有處理都可以繼續拉，但怎麼樣可以讓他直接跳出此迴圈呢?
    except StaleElementReferenceException:
        logger.info(f"{date_split} not found result, do next search")
        continue


    only_one_page = False
    try:
        # 如果沒抓到這個，很可能只有一頁，那麼下面製造pages就不做了，程式也會在這裡斷開，交給下面有個if處理，若這種情況發生就去抓原本iframe的網址(這樣一頁的內容就可以爬了)，只有一頁加入page_url_list這樣
        WebDriverWait(driver, 10).until(  #看這邊時間要不要設晚一點?
        EC.presence_of_element_located(
        (By.CSS_SELECTOR, "a#hlNext")
        ))

        page_url = driver.find_element(By.CSS_SELECTOR, 'a#hlNext').get_attribute('href')

        #計算頁數，與生成page_list
        how_many_page = driver.find_element(By.CSS_SELECTOR, "div#plPager span")
        page_re = "\d*(?=\s頁)"
        answer_page = re.search(page_re, how_many_page.text)
        page_url_list.clear()
        regex_page_url = "&page=[\d]*"
        test = re.split(regex_page_url, page_url)
        if int(answer_page[0]) <= 25:
            for each_page in range(1,int(answer_page[0])+1):
                page_url_list.append(f"{test[0]}&page={each_page}{test[1]}") 
        else:
            logger.warning(f"{date_split} has {block_cases_number} cases, need to minimize split date interval, otherwise data shortage!!")
            for each_page in range(1, 26):
                page_url_list.append(f"{test[0]}&page={each_page}{test[1]}")
                
            #這邊寫得好的話若超過25頁就要回到一開始的搜尋頁面，並把日期格式再切小
            # 妥協的辦法就是超過25就給他25頁就好

    except TimeoutException:
        only_one_page = True
        logger.info(f"{date_split} not found...next_page_button and pages or may be it has only one page")
    except StaleElementReferenceException:
        only_one_page = True
        logger.info(f"{date_split} not found...next_page_button and pages or may be it has only one page")


    if only_one_page:
        page_url_list.clear()
        page_url_list.append(one_page_url) 

    #計算一個搜尋的case數量並計算
    case_counter = 1
    for page in page_url_list:
        driver.get(page)

        
        try:
            cssSelector = 'a#hlTitle'
            WebDriverWait(driver, 30).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, cssSelector)
            ))

            case_url = driver.find_elements(By.CSS_SELECTOR, cssSelector)
            case_url_list = []
            for case in case_url:
                case_url_list.append(case.get_attribute('href'))
            
        except TimeoutException:
            # 這裡反而是抓到，沒有搜尋結果的錯誤
            logger.warning(f"{page} cases not found, may be there is no search result") #我在想這可能根本抓不到，抓到就跳出怎會到click??
            continue

        # cases找到後開始一個一個點
        # 這是每一頁當中的至多20筆的資料   
        for case_u in case_url_list:      
            driver.get(case_u)          # 這邊若有問題可以試試看改抓網址，然後用driver.get()處理

            try:
                # 看有沒有抓到js生成的側攔"當中的li"，li就是js後來生成的
                # 這邊要改可能還是沒有laws，而且結果超奇怪....根本抓不到law結果欄位有值，抓地到history但沒有值...
                
                side_li_selector_2 = 'div#JudHis div.panel-body ul li'
                WebDriverWait(driver, 15).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, side_li_selector_2)
                ))

                #還是這邊可以等兩個?等個兩個載好在來取得動態html

            except TimeoutException:
                # 如果有錯的話抓這個webelement的text(標題)，href(超連結)
                logger.info(f"{id_id}:{case_u} not found...history, may be critical error")
                driver.refresh()
                sleep(10)
                #這邊之後要改logging
            except StaleElementReferenceException:   # Except 中不能再用webdriver了，不然抓不到又抱錯(不然就是要用簡單的等待錯略，部會丟exception那種)，給他refresh，不然就直接繼續抓就好，空的就空的
                logger.info(f"{id_id}:{case_u} driver refresh to relocated")
                driver.refresh()
                sleep(10)
                    

            try:   
                # 但這個欄位真的有可能抓不到那就要抱錯，可是我們應該分開來報，因為兩個的問題不一樣
                # 這邊不只會有TimeoutException，還有其他的所以會抱錯嘗試refresh，重抓看看
                side_li_selector_1 = 'ul.rela-law li'   #這邊真的很吃網路，其實也不算錯，就是網路加仔問題而已，但好像有點難解決
                WebDriverWait(driver, 10).until(    # 我還是不懂這邊怎麼會timeoutexception，不是用try except包起來了嗎????因為這邊抓到TimeoutException之後又raise了StaleElementReferenceException，所以不是沒抓到，而是因為StaleElementReferenceException而停的
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, side_li_selector_1)
                ))

            #我之前有疑問，問什麼timeoutexception還會raise StaleElementReferenceException，這是因為我可能有點誤解timeoutexception了，timeoutexception是"網頁加載時間"上的問題，根本沒有加載完成所以沒辦法找到該元素，而StaleElementReferenceException是網頁加載"完"出現的元素定位問題(舉個例子就是，因為一些問題導致某個元素沒有出現在DOM TREE當中，所以根本找不到該元素)(在此情境中，就是"相關法條"常常跑不出來，但只要刷新頁面就可以解決了)所以當抓到StaleElementReferenceException，我就driver.refresh()，刷新看看，刷新之後也不做判斷了，就繼續往下做，沒有的欄位就空著
            # 所以WebDriverWait，TimeoutException, StaleElementReferenceException都會丟出來啦
            except TimeoutException:
                logger.info(f"{id_id}:{case_u} not found...laws, it's ok")
                driver.refresh() # 如果等不到就refresh，畢竟這邊也只是再判斷有沒有而已
                sleep(10)
            except StaleElementReferenceException:
                logger.info(f"{id_id}:{case_u} driver refresh to relocated")
                driver.refresh()
                sleep(10)


            #取得動態html #反正都要連到這個case當中，若上面兩個沒抓到，就只是side_bar的資料有少但其他資料還是有的
            #但我就在想這樣會不會很雞肋、而且反而拖到時間，因為有些case是真的沒有上面那兩個side_bar，還是乾脆就強制等待sleep算了，不知道哪種比較優且快
            sleep(1) # 稍微等一下laws那個side_bar位置
            case_html = driver.page_source
            soup = bs(case_html, "lxml")

            allSelector = 'div.col-td'
            all = soup.select(allSelector)

            laws_list = []
            linesSelector = 'ul.rela-law li'
            lines = soup.select(linesSelector)
            for law in lines:
                laws_list.append(str(law.get_text()))

            
            history_judgement = soup.select("div#JudHis div.panel-body ul li")
            history_judgement_list = []
            for index in range(0, len(history_judgement)):
                if not str(history_judgement[index].select_one('a')) == "None":
                    history_judgement_list.append({
                        "case_id":str(history_judgement[index].get_text()),
                        "url":"https://judgment.judicial.gov.tw/FJUD/"+ str(history_judgement[index].select_one('a')['href'])
                    })
                else:
                    history_judgement_list.append({
                    "case_id":str(history_judgement[index].get_text())
                    })


            all_case_list.append({
            "id_id":id_id,
            "case_id":str(all[0].get_text().strip()),
            "case_url":str(case_u),
            "date":str(all[1].get_text().strip()),
            "simple_reason":str(all[2].get_text().strip()),
            "laws":laws_list,
            "history_judgement":history_judgement_list,
            "context":str(all[3].get_text()),
            })
            
            logger.info(f"id_id:{id_id}, search {date_split}, case_number:{case_counter}/{block_cases_number} is ok")
            id_id += 1
            case_counter += 1
            if case_counter == block_cases_number:
                logger.info(f"{date_split} This block is finished")

2024-05-20 21:18:30 - INFO - [113, 7, 1, 12, 31] is search
2024-05-20 21:18:30 - INFO - [113, 7, 1, 12, 31] is search
2024-05-20 21:18:30 - INFO - [113, 7, 1, 12, 31] is search
2024-05-20 21:18:30 - INFO - [113, 7, 1, 12, 31] is search
2024-05-20 21:18:31 - INFO - search [113, 7, 1, 12, 31], it's base url is https://judgment.judicial.gov.tw/ErrorPage.aspx?frm=iframe&err=Q003
2024-05-20 21:18:31 - INFO - search [113, 7, 1, 12, 31], it's base url is https://judgment.judicial.gov.tw/ErrorPage.aspx?frm=iframe&err=Q003
2024-05-20 21:18:31 - INFO - search [113, 7, 1, 12, 31], it's base url is https://judgment.judicial.gov.tw/ErrorPage.aspx?frm=iframe&err=Q003
2024-05-20 21:18:31 - INFO - search [113, 7, 1, 12, 31], it's base url is https://judgment.judicial.gov.tw/ErrorPage.aspx?frm=iframe&err=Q003
2024-05-20 21:18:42 - INFO - [113, 7, 1, 12, 31] not found result, do next search
2024-05-20 21:18:42 - INFO - [113, 7, 1, 12, 31] not found result, do next search
2024-05-20 21:18:42 - INFO - [11

KeyboardInterrupt: 

In [ ]:
print(block_cases_number)

 60 


In [ ]:
print(all_case_list[4444]['laws'])

['刑事訴訟法 第 159、159.5、198、208、229、230、231、299 條（96.12.12）', '中華民國刑法 第 1、11、13、25、38、55、271 條（98.01.21）', '槍砲彈藥刀械管制條例 第 8、12 條（97.11.26）', '中華民國刑法施行法 第 1.1、2 條（98.01.21）']


In [ ]:
print(len(all_case_list))

9609


In [ ]:
# 就寫成dataframe再轉成csv八
tt = pd.DataFrame(all_case_list)
tt.to_csv('./all_cases_v3.csv', index=False)

In [ ]:
tt[5664:5670]

,id_id,case_id,date,simple_reason,laws,history_judgement,context
5664,5665,臺灣桃園地方法院 95 年度重訴字第 77 號刑事判決,民國 96 年 06 月 15 日,殺人,[刑事訴訟法 第 3、156、158.3、159、202、203、206、208、299 條...,[{'case_id': '臺灣桃園地方法院 95 年度 重訴 字第 77 號裁定(95.1...,\n\n\n\n\n臺灣桃園地方法院刑事判決 95年度重訴字第77號\n公 訴 ...
5665,5666,臺灣雲林地方法院 96 年度重訴字第 2 號刑事判決,民國 96 年 06 月 15 日,,[刑事訴訟法 第 8、159、159.1、159.2、159.3、159.4、159.5、1...,[{'case_id': '臺灣雲林地方法院 96 年度 重訴 字第 2 號裁定(96.04...,\n\n\n\n\n臺灣雲林地方法院刑事判決 96年度重訴字第2號\n公 訴 ...
5666,5667,臺灣彰化地方法院 96 年度訴字第 554 號刑事判決,民國 96 年 06 月 15 日,殺人未遂等,"[刑事訴訟法 第 159、159.1、159.5、299 條（96.03.21）, 中華民國...",[{'case_id': '臺灣彰化地方法院 96 年度 訴 字第 554 號裁定(96.0...,\n\n\n\n\n臺灣彰化地方法院刑事判決 96年度訴字第554號\n公 訴...
5667,5668,臺灣高等法院 高雄分院 95 年度上重訴字第 26 號刑事判決,民國 96 年 06 月 14 日,殺人等,[刑事訴訟法 第 2、159.1、159.5、181、186、369 條（96.03.21）...,[{'case_id': '臺灣高雄地方法院 94 年度 重訴 字第 60 號裁定(94.1...,\n\n\n\n\n臺灣高等法院高雄分院刑事判決 95年度上重訴字第26號\n上 訴 ...
5668,5669,臺灣臺北地方法院 96 年度訴字第 614 號刑事判決,民國 96 年 06 月 14 日,殺人未遂,"[刑事訴訟法 第 159.1、299 條（96.03.21）, 中華民國刑法 第 1、2、2...",[{'case_id': '臺灣臺北地方法院 96 年度 訴 字第 614 號裁定(96.0...,\n\n\n\n\n臺灣臺北地方法院刑事判決 96年度訴字第614號\n公 訴...
5669,5670,臺灣高等法院 臺中分院 96 年度上更(二)字第 130 號刑事判決,民國 96 年 06 月 13 日,殺人未遂等,"[刑事訴訟法 第 159.1、300、369 條（96.03.21）, 中華民國刑法 第 1...",[{'case_id': '臺灣南投地方法院 94 年度 訴 字第 540 號判決(94.1...,\n\n\n\n\n臺灣高等法院臺中分院刑事判決 96年度上更(二)字第130號\n上 ...
